<a href="https://colab.research.google.com/github/sallywang147/SCInvarinfer/blob/main/Fine_tuned_T5_VeriSol_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]
!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 1.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 878.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.4 MB/s eta 0:00:00
  Attempting uninstall: pygments
    Found existing installation: 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=6fe3c7a6eff938438bcb5cdca855e41b3e72d2a431fe67822c6d1e1301733a87
  Stored in directory: /root/.cache/pip/wheels/ea/f9/a1/34e2943cce3cf7daca304bfc35e91280694ced9194a487ce2f
Successfully built ws4py


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Importing libraries
import anvil.server
anvil.server.connect("server_DRHAH5WMRL4CDEKWGECLO4R5-FNQ76OID3FCLIPND")
import os
import numpy as np
import pandas as pd
import plotly.express as px
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console


user_params = {
      'model_path': '/content/drive/MyDrive/model_files',
      'test_contract_path': '/content/drive/MyDrive/LoopFor.sol',
      'prediction_file': '/content/drive/MyDrive/pred.txt',
      'program_path': '/content/drive/MyDrive/sample.txt'
      }
test_params = {
      'batch_size': 8,
      'shuffle': False,
      'num_workers': 0
      }

In [9]:
class TestDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, source_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.source_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    print('source_text: ', source_text)
    source = self.tokenizer.batch_encode_plus([source_text], \
                                              max_length=self.source_len, \
                                              pad_to_max_length=True, \
                                              truncation=True, \
                                              padding="max_length",\
                                              return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
    }

In [10]:
#test how the model performs on never-seen test contracts
@anvil.server.callable
def generate(saved_model_path, test_contract_path, prediction_file, \
             verify_with_boogie):
   saved_model_path = user_params['model_path']
   test_contract_path = user_params['test_contract_path']
   prediction_file = user_params['prediction_file']
   
   model = T5ForConditionalGeneration.from_pretrained(saved_model_path)
   tokenizer = T5Tokenizer.from_pretrained("t5-small")
   col = ['Test']
   f = open(test_contract_path, "r")
   file = f.read()
   df = pd.DataFrame([file], columns=col)
   col = ['Test']
   df = pd.DataFrame([file], columns=col)

   model.eval()
   test_val = TestDataSetClass(df, tokenizer, source_len=512, source_text="Test")
   test_loader = DataLoader(test_val, **test_params)
   predictions = []
   with torch.no_grad():
      for _, data in enumerate(test_loader, 0):
          ids = data['source_ids']
          mask = data['source_mask'] #.to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, \
                                    clean_up_tokenization_spaces=True) for g in generated_ids]
          print("preds: ", preds)
          if _%10==0:
              print(f'Completed {_}')
          predictions.extend(preds)
   with open(prediction_file, 'w') as writefile:
      for line in predictions:
        print(line)
        writefile.write(line)
   return predictions

In [15]:
@anvil.server.callable
def instantiate_program(instantiated_program_path):
   saved_model_path = user_params['model_path']
   test_contract_path = user_params['test_contract_path']
   prediction_file = user_params['prediction_file']
   instantiated_program_path = user_params['program_path']
   model = T5ForConditionalGeneration.from_pretrained(saved_model_path)
   tokenizer = T5Tokenizer.from_pretrained("t5-small")
   col = ['Test']
   f = open(test_contract_path, "r")
   file = f.read()
   df = pd.DataFrame([file], columns=col)
   col = ['Test']
   df = pd.DataFrame([file], columns=col)
   return file

In [16]:
file = instantiate_program(user_params['program_path'])
print(file)

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


pragma solidity >=0.4.24 <0.6.0;

contract LoopFor {

    uint[2] a;

    function testBoundedForLoop() public {
        for (uint i = 0; i < 2; i += 1) {
            a[i] = i;
        }
        assert (a[0] == 0);
        assert (a[1] == 1);
    }

    uint[10] b;
    
    function testUnboundedForLoop(uint n) public {
        require (n > 0 && n < 10);
        for (uint i = 0; i < n; i += 1) {
            b[i] = i;
        }
        for (uint j = 0; j < n; j += 1) {
            assert (b[j] == j);
        }
    }

}



In [ ]:
anvil.server.wait_forever()
#generate(df, trained_model, tokenizer)